## Using a manually- colated list of references to to search for DOIs using the Crossref API with the python library 'habanero' 
(conda installed v0.6)
check with first ten 



code for basic install and a simple search of the works using a query and a filter



In [1]:
import pandas as pd
from habanero import Crossref, exceptions
import time

### First bring in the existing list of publictaions (using the columns needed)

In [3]:
df1 = pd.read_excel('./Source_files/2019-05-07-BRC2 publications-DOI_LBedit.xlsx', sheet_name='current_list', usecols=[0,1,2,3])
df1.shape

(2378, 4)

In [4]:
df1.head(2)

,ID,DOI,complete,csv_post_title
0,1125,10.1186/s12881-014-0095-4,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",Apparently synonymous substitutions in FGFR2 a...
1,1996,10.1183/13993003.00321-2016,", Pattinson KT, Turner MR. A wider pathologic...",A wider pathological network underlying breath...


In [5]:
#make dataframe from only those missing DOIs
#df1[~df1.doi.isna()]  gives those with DOIs (1440 as reported)
df1.loc[~df1.DOI.isna()].shape


(1442, 4)

### Therefore identifying that 1440 of the 2378 publications have DOIs (934 do not)
### _however, we don't know any of them are correct_ so it will best to acquire all top matches from CrossRef


In [5]:
df1mini = df1.iloc[10:15]
df1mini.head()

,ID,DOI,complete,csv_post_title
10,168,NaN,"Pagnamenta AT, Murray JE, Yoon G, Akha ES, ...",A novel nonsense CDK5RAP2 mutation in a Somali...
11,2026,10.3201/eid2204.151812,"Pybus OG, Bonsall D, Gregory WF, Ip CL, Donfi...",Evaluation of Viremia Frequencies of a Novel H...
12,1154,10.7554/eLife.03821,"Williams JP, Hurst J, StÃ•_hr W, Robinson ...",HIV-1 DNA predicts disease progression and pos...
13,1729,10.1111/dote.12244,"., Findlay JM, Tilson RC, Harikrishnan A, Sgro...",Attempted validation of the NUn score and infl...
14,2123,10.1016/j.dib.2016.08.046,"., Rysnik O, McHugh K, van Duivenvoorde L, van...",Data showing non-conventional HLA-B27 expressi...


In [6]:
cr = Crossref()

In [7]:
#Crossref(api_key = "168421")
# set a mailto address
Crossref(mailto = "laurence.brown@ndcn.ox.ac.uk")

< Crossref 
URL: https://api.crossref.org
KEY: None
MAILTO: laurence.brown@ndcn.ox.ac.uk
>

In [8]:
#cr.filter_details()

In [9]:
'''['DOI', 'URL', 'alternative-id', 'author','indexed', 'is-referenced-by-count',
       'language', 'license', 'link',  'published-online', 'published-print', 'publisher',
       'reference', 'reference-count', 'references-count', 'relation', 'score',
       'short-container-title', 'source', 'title', 'type', 'update-to']
'''
#select=['title', 'link','URL', 'author','is-referenced-by-count','alternative-id', 'author','indexed',],

"['DOI', 'URL', 'alternative-id', 'author','indexed', 'is-referenced-by-count',\n       'language', 'license', 'link',  'published-online', 'published-print', 'publisher',\n       'reference', 'reference-count', 'references-count', 'relation', 'score',\n       'short-container-title', 'source', 'title', 'type', 'update-to']\n"

In [10]:
cr.works(query_title=df1mini.csv_post_title[12]
             ,select=['title','DOI'],
                      filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)

{'status': 'ok',
 'message-type': 'work-list',
 'message-version': '1.0.0',
 'message': {'facets': {},
  'total-results': 1765187,
  'items': [{'title': ['HIV-1 DNA predicts disease progression and post-treatment virological control'],
    'DOI': '10.7554/elife.03821'}],
  'items-per-page': 1,
  'query': {'start-index': 0, 'search-terms': None}}}

In [11]:
def CrossRefDoiTop1(df):
    try:
        search = cr.works(query_title=df.csv_post_title
             ,select=['title','DOI'],
                      filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)
        time.sleep(1)
        return search['message']['items'][0]
    except:
        return ('None')
    finally:
        time.sleep(1)
        pass

In [12]:
#df1x.columns


In [13]:
df1check= df1mini.assign(crossref_API_out =df1mini.apply(CrossRefDoiTop1, axis=1))

In [14]:
df1check

,ID,DOI,complete,csv_post_title,CrossRefDoi
10,168,NaN,"Pagnamenta AT, Murray JE, Yoon G, Akha ES, ...",A novel nonsense CDK5RAP2 mutation in a Somali...,{'title': ['High-frequency sensorineural heari...
11,2026,10.3201/eid2204.151812,"Pybus OG, Bonsall D, Gregory WF, Ip CL, Donfi...",Evaluation of Viremia Frequencies of a Novel H...,{'title': ['Evaluation of Viremia Frequencies ...
12,1154,10.7554/eLife.03821,"Williams JP, Hurst J, StÃ•_hr W, Robinson ...",HIV-1 DNA predicts disease progression and pos...,{'title': ['HIV-1 DNA predicts disease progres...
13,1729,10.1111/dote.12244,"., Findlay JM, Tilson RC, Harikrishnan A, Sgro...",Attempted validation of the NUn score and infl...,{'title': ['Attempted validation of the NUn sc...
14,2123,10.1016/j.dib.2016.08.046,"., Rysnik O, McHugh K, van Duivenvoorde L, van...",Data showing non-conventional HLA-B27 expressi...,{'title': ['Data showing non-conventional HLA-...


In [15]:
newdata = pd.DataFrame(pd.io.json.json_normalize(df1check.crossref_API_out))  

In [16]:
newdata

,DOI,title
0,10.1002/lary.25544,[High-frequency sensorineural hearing loss in ...
1,10.3201/eid2204.151812,[Evaluation of Viremia Frequencies of a Novel ...
2,10.7554/elife.03821,[HIV-1 DNA predicts disease progression and po...
3,10.1111/dote.12244,[Attempted validation of the NUn score and inf...
4,10.1016/j.dib.2016.08.046,[Data showing non-conventional HLA-B27 express...


In [17]:
df1.iloc[27]

ID                                                             1520
DOI                                    10.1371/journal.ppat.1004954
complete          Adland E, Paioni P, Thobakgale C, Laker L, Mor...
csv_post_title    Discordant impact of HLA on viral replicative ...
Name: 27, dtype: object

In [19]:
df1x1= df1[0:801]
df1x2= df1[801:1601]
df1x3= df1[1601:2379]#.reset_index()
df1x3.head(2)

,ID,DOI,complete,csv_post_title
1601,839,10.4137/BMI.S16553,"Patel S, Murphy D, Haralambieva E, Abdulla ZA,...",Increased expression of phosphorylated FADD in...
1602,2411,10.1016/j.omtn.2016.12.006,"Patricio MI, Barnard AR, Orlans HO, McClements...",Inclusion of the Woodchuck Hepatitis Virus Pos...


In [20]:
df1x1out = df1x1.assign(crossref_API_out =df1x1.apply(CrossRefDoiTop1, axis=1))
df1x1out.to_csv('bodDois1x1out.csv')

In [21]:
df1x1out.head()

,ID,DOI,complete,csv_post_title,crossref_API_out
0,1125,10.1186/s12881-014-0095-4,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",Apparently synonymous substitutions in FGFR2 a...,{'title': ['Apparently synonymous substitution...
1,1996,10.1183/13993003.00321-2016,", Pattinson KT, Turner MR. A wider pathologic...",A wider pathological network underlying breath...,{'title': ['A wider pathological network under...
2,506,NaN,"Adib-Samii P, Rost N, Traylor M, Devan W, ...",17q25 Locus is associated with white matter hy...,{'title': ['17q25 Locus Is Associated With Whi...
3,1430,10.1093/annonc/mdu449,"and I. Tomlinson*, Findlay JM, Middleton MR, ...",A systematic review and meta-analysis of somat...,{'title': ['A systematic review and meta-analy...
4,848,NaN,"Dichgans M, Malik R, KÃ•_nig IR, Rosand J,...",Shared genetic susceptibility to ischemic stro...,{'title': ['Shared Genetic Susceptibility to I...


In [22]:
df1x2out = df1x2.assign(crossref_API_out =df1x2.apply(CrossRefDoiTop1, axis=1))
df1x2out.to_csv('bodDois1x2out.csv')

In [23]:
df1x2out.head()

,ID,DOI,complete,csv_post_title,crossref_API_out
801,969,10.2337/dc13-2539,"Guest JF, Panca M, Sladkevicius E, Taheri S, S...",Clinical outcomes and cost-effectiveness of co...,{'title': ['Obstructive sleep apnea in childre...
802,1767,10.1182/blood-2015-05-647578,"GuiÃ¨ze R, Robbe P, Clifford R, de Guibert S, ...",Presence of multiple recurrent mutations confe...,{'title': ['Presence of multiple recurrent mut...
803,1935,10.1038/nature14347,"Gundem G, Van Loo P, Kremeyer B, Alexandrov LB...",The evolutionary history of lethal metastatic ...,{'title': ['Treatment of metastatic prostate c...
804,2060,10.1093/jac/dkw177,"Guo Q, Tomich AD, McElheny CL, Cooper VS, Stoe...",Glutathione-S-transferase FosA6 of Klebsiella ...,{'title': ['Glutathione-S-transferase FosA6 of...
805,1690,10.1158/1055-9965.EPI-15-0363,"Guo X, Long J, Zeng C, Michailidou K, Ghoussai...",Fine-Scale Mapping of the 4q24 Locus Identifie...,{'title': ['Fine-Scale Mapping of the 4q24 Loc...


In [24]:
df1x3out = df1x3.assign(crossref_API_out =df1x3.apply(CrossRefDoiTop1, axis=1))
df1x3out.to_csv('bodDois1x3out.csv')

In [25]:
# Bring these smaller dataframes together 
df1x_out = pd.concat([df1x1out, df1x2out, df1x3out])

In [124]:
#Save the work so we don't need to get the data again

df1x_out.to_csv('./A1out_CrossRef_BRC_Bodlist_BODandCrossRefDois_05Aug19.csv')
df1x_out.to_json('./A1out_CrossRef_BRC_Bodlist_BODandCrossRefDois_05Aug19.json', orient='records', lines=True)
df1x_out.crossref_API_out.to_json('./A1out_CrossRef_API_data_alone_05Aug19.json', orient='index')
df1x_out.head()


,ID,DOI,complete,csv_post_title,crossref_API_out
0,1125,10.1186/s12881-014-0095-4,"&amp; , Fenwick AL, Goos JAC, Rankin J, Lord ...",Apparently synonymous substitutions in FGFR2 a...,{'title': ['Apparently synonymous substitution...
1,1996,10.1183/13993003.00321-2016,", Pattinson KT, Turner MR. A wider pathologic...",A wider pathological network underlying breath...,{'title': ['A wider pathological network under...
2,506,NaN,"Adib-Samii P, Rost N, Traylor M, Devan W, ...",17q25 Locus is associated with white matter hy...,{'title': ['17q25 Locus Is Associated With Whi...
3,1430,10.1093/annonc/mdu449,"and I. Tomlinson*, Findlay JM, Middleton MR, ...",A systematic review and meta-analysis of somat...,{'title': ['A systematic review and meta-analy...
4,848,NaN,"Dichgans M, Malik R, KÃ•_nig IR, Rosand J,...",Shared genetic susceptibility to ischemic stro...,{'title': ['Shared Genetic Susceptibility to I...


In [77]:
df1x_out[df1x_out.crossref_API_out.isna()]

,ID,DOI,complete,csv_post_title,crossref_API_out
